# DRF란 (Django REST Framework)

- 장고를 기반으로 REST API 서버를 만들기 위한 라이브러리
- API는 웹 뿐만 아니라 앱이나 다양한 플랫폼의 백엔드 서비스를 위해 JSON과 같은 규격화된 데이터를 제공
    - DRF를 사용하면 기존의 자체적인 웹템플릿에만 데이터를 전덜하던 장고 프로젝트에서 JSON과 같은 양식으로 다양한 플랫폼의 클라이언트에게 데이터를 제공해줄 수 있는 API 서버 프로젝트가 완성됨

In [1]:
import requests

In [20]:
res = requests.get("http://127.0.0.1:8000/example/cbv/book/2/")

In [21]:
res

<Response [200]>

In [22]:
res.text

'{"bid":2,"title":"AI 2024","author":"김덕진","category":"프로그래밍","pages":416,"price":19800,"published_date":"2023-10-25","description":"트렌드&활용백과"}'

In [23]:
res.json()

{'bid': 2,
 'title': 'AI 2024',
 'author': '김덕진',
 'category': '프로그래밍',
 'pages': 416,
 'price': 19800,
 'published_date': '2023-10-25',
 'description': '트렌드&활용백과'}

In [11]:
res.json()[0]["title"]

'코딩 테스트 합격자 되기 - C++ 편'

In [24]:
res = requests.post("http://127.0.0.1:8000/example/cbv/books/",
                   data = {
    "bid" : 3,
    "title" : "최소한의 한국사",
    "author" : "최태성",
    "category" : "역사",
    "pages" : 352,
    "price" : 19800,
    "published_date" : "2023-06-21",
    "description" : "5천 년 역사가 단숨에 이해되는"
})

In [ ]:
# 수정
requests.put()

# 삭제
requests.delete()

In [25]:
res = requests.delete("http://127.0.0.1:8000/example/mixin/book/2/")

In [26]:
res

<Response [204]>

In [27]:
res.text

''

In [ ]:
# pip install djangorestframework
# django-admin startproject myapi(프로젝트명) .
# python manage.py startapp example(앱 이름)

In [1]:
# Settings = py

INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "example",
    "rest_framework", # django rest framework를 쓸거야!
]

TIME_ZONE = "Asia/Seoul"

In [ ]:
# python manage.py makemigrations
# python manage.py migrate - db.sqlite3가 만들어짐~

In [ ]:
# example/views.py

from rest_framework import status
from rest_framework.response import Response
from rest_framework.views import APIView
from rest_framework.decorators import api_view

# Create your views here.
@api_view(["GET"]) # decorator 데코레이터
def helloAPI(request):
    return Response("hello world!")

## DRF의 뷰

- @와 함께 작성된 코드를 데코레이터라고 부름
    - 데코레이터는 함수를 꾸미는 역할
    - 해당 함수에 대한 성격이나 스타일을 표시해주는 표기법
    - helloAPI는 get요청을 받을 수 있는 api라는 것을 api_view라는 표기법으로 나타냄
    
- request 객체는 요청에 대한 정보를 담고 있음
    - 요청이 어떤 타입인지(get, post)
    - 사용자가 어떤 데이터를 함께 보내주었는지
    - 이런 정보를 알고 싶을 때 request.method, request.data 등으로 요청타입과 데이터에 접근할 수 있음
    
- Response 클래스는 DRF의 결과 반환 방식
    - request와 마찬가지로 Response에는 응답에 대한 정보를 담고 있음
    - response.data, response.status

In [ ]:
# myapi/urls.py

from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path("admin/", admin.site.urls),
    path("example/", include("example.urls")),
]

In [ ]:
# example/urls.py

from django.urls import path
from example.views import helloAPI

urlpatterns = [
    path("hello/", helloAPI),
]

## Django와 달라진 점

- DRF는 Response를 제공하는 API의 형태로 결과물이 나옴
    - 템플릿의 형태가 아닌 JSON과 같은 형태의 응답을 제공
    - 기존 장고에서는 템플릿으로 데이터를 제공했다면 DRF에서는 Serializer가 템플릿과 같은 역할을 수행
    
| 특징 | Pure Django | Django REST Framework |
| :--: | :--: | :--: |
| 개발 목적 | 웹 풀스택 개발 | 백엔드 API 서버 개발 |
| 개발 결과 | 웹 페이지를 포함한 웹 서비스 | 여러 클라이언트에서 사용할 수 있는 API 서버 |
| 응답 형태 | HTML | JSON|
| 다른 파일 | templates | serializers.py |

# Django REST Framework 기초 개념

## DRF Serializer

- 시리얼라이저의 사전적 의미는 직렬화
    - 직렬화는 장고 프로젝트에서 만든 모델을 JSON으로 변환하는 것
    
- DRF 내에서 데이터를 저정할 때에는 장고의 모델을 통해 저장
    - 모델은 데이터베이스 테이블을 추상화한 개념
    - 장고의 ORM을 통해 파이썬 문법으로 데이터를 처리할 수 있음
    - 이 때 장고 모델의 데이터는 파이썬 객체의 형태
    
- API는 위의 데이터를 클라이언트에 보내주는 역할을 하는데 파이썬 객체를 그대로 보낸다면 파이썬 데이터를 읽지 못할 수도 있음
    - 그렇기 때문에 파이썬 데이터를 읽을 수 있도록 문자열(JSON 등)의 형태로 변환해서 보내줘야함
    - 파이썬 데이터 객체를 문자열 등으로 변환하는 작업을 직렬화(serializer)
    
- 반대로 클라이언트가 데이터를 DRF 서버에 보내주는 경우
    - 클라이언트는 API 요청에 데이터를 JSON 등 문자열 형태로 입력하여 보내주게 됨
    - DRF 서버에서는 모델을 통해 데이터를 저장하려면 데이터가 파이썬 객체의 형태여야 함
    - 따라서 앞선 경우와 반대로 JSON 등 문자열 데이터를 파이썬 데이터 객체로 변환해야 함
        - 이 작업을 역직렬화(Deserialize)라고 함
        
- 시리얼라이저는 직렬화와 역직렬화 기능을 동시에 가짐
    - 요약하자면 시리얼라이저는 클라이언트와 API 서버 간 데이터 양식을 맞춰주는 변환기

In [ ]:
# example / models.py

class Book(models.Model):
    bid = models.IntegerField(primary_key = True) # 책 id
    title = models.CharField(max_length = 50) # 책 제목
    author = models.CharField(max_length = 50) # 저자
    category = models.CharField(max_length = 50) # 카테고리
    pages = models.IntegerField() # 페이지 수
    price = models.IntegerField() # 가격
    published_date = models.DateField() # 출판일
    description = models.TextField() # 도서 설명
    
> python manage.py makemigrations
> python manage.py migrate

## FBV, CBV

- 장고에서는 크게 두 가지 유형으로 뷰를 개발할 수 있음
    - 함수 기반 뷰(Function Based View; FBV) # 복잡한 애들
    - 클래스 기반 뷰(Class Based View; CBV) # 간단한 애들
    
- 뷰를 작성하는 방식의 차이일 뿐 기능 상 차이는 없음
    - 지금까지 작성해온 뷰는 모두 FBV
    
- FBV와 CBV를 공통적으로 도와주는 도구가 APIView
    - FBV에서는 @api_view와 같이 데코레이터 형태로 APIView를 사용
    - CBV에서는 APIVied라는 클래스를 상속받는 클래스의 형태로 사용

In [ ]:
# exalmple / views.py

# 클래스형 뷰로 HelloAPI 작성
class HelloAPI(APIView):
    def get(self, request, format = None): # (인스턴스 개체, 요청 정보, 정보)
        return Response("hello world!")

In [ ]:
# example / urls.py

from django.urls import path
from example.views import helloAPI, HelloAPI

urlpatterns = [
    path("hello/", helloAPI),
    path("cbv/hello/", HelloAPI.as_view()),
]


In [ ]:
# example / views.py

@api_view(["GET", "POST"]) # get/post 요청을 처리하게 만들어주는 데코레이터
def booksAPI(request):
    if request.method == "GET": # requst와 method가 get 요청이 들어온다면 도서 전체 정보를 제공해줄 것
        books = Book.objects.all() # book 모델로부터 전체 데이터 가져오기
        
        # Seriailizer에 전체 데이터를 집어넣기(직렬화) # 직렬화는 보통 단일 데이터를 받음!
        serializer = BookSerializer(books, many = True) # many = True = 전체 데이터를 받았기 때문에 many = True!
        return Response(serializer.data, status=status.HTTP_200_OK) # status.HTTP_200_OK =  응답코드!
    
    elif request.method == "POST": # post 요청(도서 정보 등록)
        # post 요청으로 들어온 데이터를 모델이 받아들일 수 있게 시리얼라이저에 집어넣기
        serializer = BookSerializer(data = request.data)

        if serializer.is_valid(): # 유효한 데이터라면
            # 유효한 데이터라면, 시리얼라이저의 역질렬화를 통해 save(), 모델 시리얼라이저의 기본 함수인, create() 함수가 동작
            serializer.save()

            # 201 메시지를 보내며 성공
            return Response(serializer.data, status=status.HTTP_201_CREATED)
            # serializer.data를 받고 상태 응답 메시지 내보내기

        # 400 잘못된 요청
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    
@api_view(["GET"])
def bookAPI(request, bid):
    book = get_object_or_404(Book, bid=bid) # bid = pk인 데이터를 Book에서 가져오고, 없으면 404에러
    serializer = BookSerializer(book) # 시리얼라이저에서 데이터를 집어넣기(직렬화)
    return Response(serializer.data, status=status.HTTP_200_OK)

# 위와 같은 뷰를 클래스형으로 작성
class Books(APIView):
    def get(self, request):
        books = Book.objects.all()
        serializer = BookSerializer(books, many=True)
        return Response(serializer.data, status=status.HTTP_200_OK)
    
    def post(self, request):
        serializer = BookSerializer(data=request.data)

        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    
class BookAPI(APIView):
    def get(self, request, bid):
        book = get_object_or_404(Book, bid=bid)
        serializer = BookSerializer(book)
        return Response(serializer.data, status=status.HTTP_200_OK)

- booksAPI는 하나의 함수에서 get(조회)인지 post(수정)인지에 따라 다르게 처리함
    - get은 도서 전체 정보를 가져오므로 모델로부터 데이터를 가져와 시리얼라이저를 통해 직렬화
    - 이 때, Book.objects.all() 명령어의 결과가 여러 데이터일 수 있기 때문에 many=True 옵션을 설정
    - post 요청에 대해서는 요청으로 들어온 데이터를 역질렬화 하여 모델에 넣어야 하므로 시리얼라이저에 request.data를 넣음
    
- bookAPI는 특정 bid의 책 데이터를 가져옴
    - 함수의 인자로 id를 넘겨받아 이를 모델에서 찾음
    - 찾은 데이터를 반환

In [ ]:
{
    "bid" : 1,
    "title" : "코딩 테스트 합격자 되기 - C++ 편",
    "author" : "박경록",
    "category" : "프로그래밍",
    "pages" : 872,
    "price" : 40500,
    "published_date" : "2024-05-01",
    "description" : "프로그래머스 제공 100 문제로 완벽 대비"
}

In [ ]:
{
    "bid" : 2,
    "title" : "AI 2024",
    "author" : "김덕진",
    "category" : "프로그래밍",
    "pages" : 416,
    "price" : 19800,
    "published_date" : "2023-10-25",
    "description" : "트렌드&활용백과"
}

In [ ]:
{
    "bid" : 3,
    "title" : "최소한의 한국사",
    "author" : "최태성",
    "category" : "역사",
    "pages" : 352,
    "price" : 19800,
    "published_date" : "2023-06-21",
    "description" : "5천 년 역사가 단숨에 이해되는"
}

## DRF의 다양한 뷰

- DRF로 API로 개발할 때 만들어야 하는 기능은 크게 5가지
    - 전체 데이터 가져오기(get)(list)
    - 1개 정보 등록하기 (post)(create)
    - 1개 정보 가져오기 (get)(retrieve)
    - 1개 정보 수정하기 (put)(update)
    - 1개 정보 삭제하기 (delete)(destroy)
    
- 이 5가지 기능을 만드는 방법은 일일이 각 메소드별로 나눠 처리하도록 작성하는 것
    - 위의  기능들을 최대한 편하고 쉽게 만들기 위해 발전된 방법이 mixins, generics, Viewset

In [ ]:
# example / views.py

from rest_framework import status, generics, mixins

class BooksAPIMixins(mixins.ListModelMixin, mixins.CreateModelMixin, generics.GenericAPIView):
    queryset = Book.objects.all() # 조회할 데이터
    serializer_class = BookSerializer # 우리 api에서 사용할 시리얼라이저 설정
    
    def get(self, request, *args, **kwargs): # get 메소드 처리 함수(전체 목록 조회)
        return self.list(request, *args, **kwargs) # mixins.ListModelMixin과 연결
    
    def post(self, request, *args, **kwargs): # post 메소드 처리 함수(1권 등록)
        return self.create(request, *args, **kwargs) # mixins.CreatModelMixin과 연결
    
class BookAPIMixins(mixins.RetrieveModelMixin, mixins.UpdateModelMixin, mixins.DestroyModelMixin, generics.GenericAPIView):
    queryset = Book.objects.all()
    serializer_class = BookSerializer
    lookup_field = "bid" # 장고 기본 모델 pk가 아닌 bid를 pk로 사용하고 있기 때문에 lookup_field로 설정

    def get(self, request, *args, **kwargs): # get 메소드 처리 함수(1권 등록)
        return self.retrieve(request, *args, *kwargs) # mixins.RetrieveModelMixin과 연결
    # RetrieveModel = (검색) 한 번만 찾는 것

    def put(self, request, *args, **kwargs): # put 메소드 처리 함수(1권 수정)
        return self.update(request, *args, **kwargs) # mixins.UpdateModelMixin과 연결
    
    def delete(self, request, *args, **kwargs): # delete 메소드 처리 함수(1권 삭제)
        return self.destroy(request, *args, **kwargs) # mixin.DestroyModelMixin과 연결

In [ ]:
# exapmle/urls.py

from django.urls import path
from example.views import helloAPI, HelloAPI, booksAPI, bookAPI, BookAPI, BooksAPI, BooksAPIMixins, BookAPIMixins

urlpatterns = [
    path("hello/", helloAPI),
    path("cbv/hello/", HelloAPI.as_view()),
    path("fbv/books/", booksAPI), # 함수형 뷰의 booksAPI와 연결
    path("fbv/book/<int:bid>/", bookAPI), # 함수형 뷰의 bookAPI와 연결
    path("cbv/books/", BooksAPI.as_view()),
    path("cbv/book/<int:bid>/", BookAPI.as_view()),
    path("mixin/books/", BooksAPIMixins.as_view()),
    path("mixin/book/<int:bid>/", BookAPIMixins.as_view()),
]

## DRF generics

- mixins 방법에는 mixins를 상속받는데 한 번에 2 ~ 3개씩 상속을 받아야 하는 번거로움이 있음
    - 따라서 DRF에서는 mixins를 조합해서 미리 generics를 만들어줌
    
- genercis의 조합 종류
    - 전체목록(genercis.ListAPIView)
    - 생성 (generics.CreateAPIView)
    - 1개 조회(generics.RetrieveAPIView)
    - 1개 수정(generics.UpdateAPIView)
    - 1개 삭제(genercis.DestroyAPIView)
    - 전체목록 + 생성(generics.ListCreateINiew)
    - 1개 조회 + 1개 수정(genercis.RetrieveUpdateAPIView)
    - 1개 조회 + 1개 삭제(generics.RetrieveDestroyAPIView)
    - 1개 조회 + 1개 수정 + 1개 삭제(genercis.RetrieveUpdateDestroyAPIView)

In [ ]:
# example/views.py

from rest_framework import status, generics, mixins

# 전체목록 + 생성 조합을 사용해서 기능 구현
class BooksAPIGenerics(generics.ListCreateAPIView):
    query = Book.objects.all()
    serializer_class = BookSerializer

# 1개 조회 + 1개 수정 + 1개 삭제 조합을 사용해서 기능 구현
class BooksAPIGenerics(generics.RetrieveUpdateDestroyAPIView):
    queryset = Book.objects.all()
    serializer_class = BookSerializer
    lookup_field = "bid"

In [ ]:
# example/urls.py

from django.urls import path
from example.views import helloAPI, HelloAPI, booksAPI, bookAPI, BookAPI, BooksAPI, BooksAPIMixins, BookAPIMixins, BooksAPIGenerics, BookAPIGenerics

urlpatterns = [
    path("hello/", helloAPI),
    path("cbv/hello/", HelloAPI.as_view()),
    path("fbv/books/", booksAPI), # 함수형 뷰의 booksAPI와 연결
    path("fbv/book/<int:bid>/", bookAPI), # 함수형 뷰의 bookAPI와 연결
    path("cbv/books/", BooksAPI.as_view()),
    path("cbv/book/<int:bid>/", BookAPI.as_view()),
    path("mixin/books/", BooksAPIMixins.as_view()),
    path("mixin/book/<int:bid>/", BookAPIMixins.as_view()),
    path("generic/books/", BooksAPIGenerics.as_view()),
    path("genecir/book/<int:bid>/", BookAPIGenerics.as_view()),
]

## DRF Viewset & Router

- 지금까지 작업했던 것은 하나의 클래스가 하나의 URL을 담당하는 방식
    -  URL마다 클래스를 만들고 각 클래스에서는 해당 URL로 들어오는 다양한 메소드를 처리할 수 있도록 하였음
    
- 따라서 queryset 이나 serializer_class 부분이 겹치게 됨
    - 하나의 클래스로 하나의 모델을 전부 처리해줄 수 있다면 겹치는 부분이 없어짐
    - 이를 위해서 Viewset을 사용
    
- Viewset과 Router 사용의 장점
    - 하나의 클래스로 하나의 모델에 대한 내용을 전부 작성할 수 있으며, 그에 따라 queryset이나 serializer_class 등 겹치는 부분을 최소화 할 수 있음
    - 라우터를 통해 URL을 일일이 지정하지 않아도 일정한 규칙의 URL을 만들 수 있음

In [ ]:
# example/views.py
from rest_framework import status, generics, mixins, viewsets

class BookViewSEt(viewsets.ModelViewSet):
    queryset = Book.objects.all()
    serializer_class = BookSerializer

- ModelViewSet을 가져와 클래스를 만들면 queryseyt과 serializer_class를 설정해주는 것으로 모델에 대한 기본적인 REST API가 완성됨
- ModelViewSet은 내부적으로 CreateModelMixin, RetrieveModelMixin, UpdateModelMixin, DestroyModelMixin, ListModelMixin을 사용함

- ViewSet은 URL과 연결할 때 라우터를 사용함

In [ ]:
# example /urls/py

from django.urls import path
from example.views import (helloAPI, HelloAPI, booksAPI, bookAPI, BookAPI, BooksAPI, 
                           BooksAPIMixins, BookAPIMixins, BooksAPIGenerics, BookAPIGenerics, 
                           BookViewSet)

from rest_framework import routers

# urlpatterns = [
#     path("hello/", helloAPI),
#     path("cbv/hello/", HelloAPI.as_view()),
#     path("fbv/books/", booksAPI), # 함수형 뷰의 booksAPI와 연결
#     path("fbv/book/<int:bid>/", bookAPI), # 함수형 뷰의 bookAPI와 연결
#     path("cbv/books/", BooksAPI.as_view()),
#     path("cbv/book/<int:bid>/", BookAPI.as_view()),
#     path("mixin/books/", BooksAPIMixins.as_view()),
#     path("mixin/book/<int:bid>/", BookAPIMixins.as_view()),
#     path("generic/books/", BooksAPIGenerics.as_view()),
#     path("generic/book/<int:bid>/", BookAPIGenerics.as_view()),
# ]

router = routers.SimpleRouter()
router.register("books", BookViewSet)

urlpatterns = router.urls

# 정리

- Mixins와 generics, Viewset & Router 까지 점점 코드가 짧아지고 DRF가 대신 만들어주는 기능들이 많아짐

    - 개발자가 할 일이 적어진다는 것은 장점이지만 개발자의 자유도가 낮아진다는 것은 단점
        - 어떤 기능을 수정할 일이 생길 때 어려움을 겪을 수 있음
    - 작은 프로젝트에서는 오히려 과하게 느껴질 수 있음
    
- 특정한 방식이 반드시 정답인 것은 아님
    - 상황에 따라 적절히 잘 활용할 수 있어야 하고 백엔드 구조가 아직 익숙하지 않다면 일일이 구현하는 것을 추천